# 09. Random Forest Model

**Mô hình:** Random Forest Regression

**Thư viện:** sklearn.ensemble.RandomForestRegressor

**Metrics:** RMSFE, MAPE

In [ ]:
# Import libraries

In [ ]:
# Load processed data

In [ ]:
# Random Forest model with GridSearchCV

In [ ]:
# Feature importance

In [ ]:
# Evaluation

In [ ]:
# Save results